In [1]:
from Dental_Tool.Data_processing import *
from Dental_Tool.Dental_Model import *
from Dental_Tool.Process_results import *
from Dental_Tool.Dataloader import *
from Dental_Tool.KFold_v3 import *
from keras.layers import *
import keras 

Using TensorFlow backend.


In [18]:
image_size = 256
input_shape = (image_size, image_size, 1)
batch_size = 100
kernel_size = 3
filters = 16
latent_dim = 128 # 隐变量取2维只是为了方便后面画图
epochs = 30
num_classes = classes = 3

In [20]:
class conv_AE(keras.Model):
    def __init__(self, input_shape):
        super(conv_AE, self).__init__()
        self.encoder = Sequential(
            [
                InputLayer(input_shape=input_shape),
                Conv2D(16, (3, 3), activation='relu', strides=2, padding='same'),
                Conv2D(8, (3, 3), activation='relu', strides=2, padding='same')
            ]
        )
        
        self.decoder = Sequential(
            [
                InputLayer(input_shape=input_shape),
                Conv2DTranspose(filters=filters, kernel_size=kernel_size, activation='relu', strides=2,padding='same'),
                Conv2DTranspose(filters=filters // 2 , kernel_size=kernel_size, activation='relu', strides=2,padding='same'),
                Conv2D(1, (3, 3), activation = "sigmoid", padding = "same")
            ]
        )
        
        self.conv_AE = Model(inputs = self.encoder.input, outputs = self.decoder(self.encoder.output))
        
    def call(self, input_tensor):

        latent_space = self.encoder.output
        reconstruction = self.decoder(latent_space)
        conv_AE = Model(inputs = self.encoder.input, outputs = reconstruction)
        
        return conv_AE(input_tensor) ##拿這個去串連所有
    
    def summary(self):
        return self.conv_AE.summary()

conv_AE = conv_AE(input_shape)        

conv_AE.compile(optimizer='Adam', loss='binary_crossentropy')
conv_AE.summary()
# conv_AE.fit(train_images, train_images, epochs=100, batch_size=128, shuffle=True,
#                 validation_data=(test_images, test_images), verbose = 2)


Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 128, 128, 16)      160       
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 64, 64, 8)         1160      
_________________________________________________________________
sequential_18 (Sequential)   multiple                  1393      
Total params: 2,713
Trainable params: 2,713
Non-trainable params: 0
_________________________________________________________________
